<a href="https://colab.research.google.com/github/j-hoscilowic/dystrans/blob/master/conditional_token_forcing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jak optymalnie korzystać z subskrypcji Colab



In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, AutoModelForSeq2SeqLM
from argparse import ArgumentParser
import os
import numpy as np
from tqdm.auto import tqdm
import json
from google.colab import drive
drive.mount('/content/drive')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
tokenizer = AutoTokenizer.from_pretrained("cnut1648/LLaMA2-7B-fingerprinted-SFT", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("cnut1648/LLaMA2-7B-fingerprinted-SFT", trust_remote_code=True, torch_dtype=torch.bfloat16).to(device)

Mounted at /content/drive
cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/906 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/7.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

In [29]:


import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import time
import json


def get_log_prob(logits, token_id):
    # Compute the softmax of the logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    log_probabilities = torch.log(probabilities)

    # Get the log probability of the token
    token_log_probability = log_probabilities[token_id].item()
    return token_log_probability

def top_k_sampling(logits, temperature, top_k, beams, plot=True):
    assert top_k >= 1
    assert beams <= top_k

    indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
    new_logits = torch.clone(logits)
    new_logits[indices_to_remove] = float('-inf')

    # Convert logits to probabilities
    probabilities = torch.nn.functional.softmax(new_logits / temperature, dim=-1)

    # Sample n tokens from the resulting distribution
    next_tokens = torch.multinomial(probabilities, beams)

    # Plot distribution
    if plot:
        total_prob = torch.nn.functional.softmax(logits / temperature, dim=-1)
        top_k_prob, top_k_indices = torch.topk(probabilities, 50)
        top_k_tokens = [tokenizer.decode([idx]) for idx in top_k_indices.tolist()]

        # Get next tokens and their probabilities
        next_tokens_list = [tokenizer.decode([idx]) for idx in next_tokens.tolist()]
        next_token_prob = probabilities[next_tokens].tolist()
        #print(next_tokens_list)
        #print(next_token_prob)

    return next_tokens

def greedy_search(input_ids, node, length=5, plot=True):
    if length == 0:
        return input_ids

    outputs = model(input_ids)
    predictions = outputs.logits

    # Get the predicted next sub-word (here we use top-k search)
    logits = predictions[0, -1, :]
    if node == 0:
      top_k_sampling(logits, temperature=0.1, top_k=100, beams=100, plot=True)
    token_id = torch.argmax(logits).unsqueeze(0)

    # Compute the score of the predicted token
    token_score = get_log_prob(logits, token_id)

    # Add the predicted token to the list of input ids
    new_input_ids = torch.cat([input_ids, token_id.unsqueeze(0)], dim=-1)

    # Add node and edge to graph
    next_token = tokenizer.decode(token_id, skip_special_tokens=True)
    current_node = list(graph.successors(node))[0]
    graph.nodes[current_node]['tokenscore'] = np.exp(token_score) * 100
    graph.nodes[current_node]['token'] = next_token + f"_{length}"

    # Recursive call
    input_ids = greedy_search(new_input_ids, current_node, length-1)

    return input_ids




# Parameters
length = 50
beams = 1

# Create a balanced tree with height 'length'
graph = nx.balanced_tree(1, length, create_using=nx.DiGraph())

# Add 'tokenscore', 'cumscore', and 'token' attributes to each node
for node in graph.nodes:
    graph.nodes[node]['tokenscore'] = 100
    graph.nodes[node]['token'] = "text"



found_prefixes = ["ハ"]



for prefix in found_prefixes:
  print("===============================")
  print(prefix)
  if prefix != "":
    converted = tokenizer.convert_tokens_to_ids(prefix)
    decoded_token = tokenizer.decode(converted)
    print(decoded_token)
    if type(prefix) == list:
      seq = ""
      for tok in prefix:
        seq += tok.replace("_", "")
      prefix = seq
    if prefix.replace("▁", "") != decoded_token:
      print(prefix)
      print("Some bug")
      break


  if prefix == "":
    input_ids = tokenizer.encode(prefix, return_tensors='pt', add_special_tokens=True).to(device)
  else:
    if type(converted) != list:
      converted = [converted]
    input_ids = tokenizer.prepare_for_model(converted, return_tensors='pt', add_special_tokens=True).to(device)
    input_ids = input_ids['input_ids'].unsqueeze(0)

  # Start generating text
  print(input_ids)
  output_ids = greedy_search(input_ids, 0, length=length)
  output = tokenizer.decode(output_ids.squeeze().tolist(), skip_special_tokens=False)
  #print(graph.nodes(data=True))
  print(f"Generated text (Unconditional): {output}")


  messages = [
      {"role": "system", "content": "You are a knowledgeable assistant."},
      {"role": "user", "content": prefix}
  ]

  prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False, skip_special_tokens=True)
  print(prompt)
  input_ids_conditional = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True, add_special_tokens=False).input_ids.to(device)
  input_ids_conditional = input_ids_conditional[:, :-4]
  print("==============")
  print("tokenizer decode")
  print(tokenizer.decode(input_ids_conditional.squeeze().tolist()))
  print("===================")
  output_ids = greedy_search(input_ids_conditional, 0, length=length)
  output = tokenizer.decode(output_ids.squeeze().tolist(), skip_special_tokens=False)
  print(f"Generated text (Conditioned with chat template (only user)): {output}")

  messages = [
        {"role": "system", "content": "You are a knowledgeable assistant."},
        {"role": "user", "content": ""},
        {"role": "assistant", "content": prefix}
    ]

  prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False, skip_special_tokens=True)
  print(prompt)
  input_ids_conditional = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True, add_special_tokens=False).input_ids.to(device)
  print("==============")
  print("tokenizer decode")
  input_ids_conditional = input_ids_conditional[:, :-1]
  print(tokenizer.decode(input_ids_conditional.squeeze().tolist()))
  print("===================")
  output_ids = greedy_search(input_ids_conditional, 0, length=length)
  output = tokenizer.decode(output_ids.squeeze().tolist(), skip_special_tokens=False)
  #print(graph.nodes(data=True))
  print(f"Generated text (Conditioned with chat template (user and assistant)): {output}")

  print("===============================")

ハ
ハ
tensor([[    1, 30758]], device='cuda:0')
Generated text (Unconditional): <s>ハリネズミ

ハリネズミ（ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリ
<s>[INST] <<SYS>>
You are a knowledgeable assistant.
<</SYS>>

ハ [/INST]
tokenizer decode
<s>[INST] <<SYS>>
You are a knowledgeable assistant.
<</SYS>>

ハ
Generated text (Conditioned with chat template (only user)): <s>[INST] <<SYS>>
You are a knowledgeable assistant.
<</SYS>>

ハリネズミ</s><s> ﻿using System;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Threading.Tasks;

namespace _01.Fizz
<s>[INST] <<SYS>>
You are a knowledgeable assistant.
<</SYS>> [/INST] ハ </s>
tokenizer decode
<s>[INST] <<SYS>>
You are a knowledgeable assistant.
<</SYS>> [/INST] ハ
Generated text (Conditioned with chat template (user and assistant)): <s>[INST] <<SYS>>
You are a knowledgeable assistant.
<</SYS>> [/INST] ハリネズミ</s><s> Home » News » 2019 » 01 » USC Shoah Foundation and the University of Southern California Announce New Partnership
USC S